In [ ]:
# pip install -r dependencies.txt

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

In [ ]:
# Carregar dados CSV
diplomatas = pd.read_csv('resources/aed_alocacao_recursos - diplomatas.csv')
cidades = pd.read_csv('resources/aed_alocacao_recursos - cidades.csv')

print(diplomatas.info())
print(cidades.info())

In [ ]:
def get_posto_trabalho(id):
  return cidades.loc[cidades['posto'] == id].to_dict(orient='records')[0]

get_posto_trabalho('P1')

In [ ]:
def calcular_custo_aresta(diplomata, posto):
    print(f"Calculando custo da aresta {diplomata.id} -> {posto.posto}")
    # Se tiver 12 anos fora, deve retornar
    if diplomata.tempo_fora >= 144:
        if posto.classificacao == '*':
            return 0.0
        else:
            return 999999999.99
    # Se tiver 2 anos na lotação atual, deve trocar de lotacao
    if diplomata.tempo_na_lotacao >= 24 and diplomata.lotacao == posto.posto:
        return 999999999.99

    # verificar se a origem á compatível com o destino. Ex. a só pode ir para B, C ou D
    if diplomata.pedagio == 0:
        posto_atual = get_posto_trabalho(diplomata.lotacao)
        # print('Posto atual {}'.format(posto_atual))
        if diplomata.lotacao != posto.posto and posto.classificacao == 'A':
            if posto_atual.get('classificacao')  == 'A':
                return 999999999.99
    elif diplomata.pedagio < 24 and posto.classificacao == 'A':
        # Se estiver a menos de 2 anos no Brasil, não pode ir para um posto 'A'
        return 999999999.99

    if posto.classificacao == 'A':
        return 0.0

    if posto.classificacao == 'B':
        return 10.0

    if posto.classificacao == 'C':
        return 100.0

    if posto.classificacao == 'D':
        return 1000.0
    
    return 999999999.99

In [ ]:
def alocar_diplomatas(matriz_custos):
  """
  Resolve o problema de alocação usando o algoritmo húngaro.

  :param matriz_custos: Matriz de custos (2D list ou numpy array)
  :return: Tupla (lista de alocações, custo total)
  """
  # Usando a função `linear_sum_assignment` da biblioteca scipy
  matriz_custos = np.array(matriz_custos)
  linhas, colunas = linear_sum_assignment(matriz_custos)
  custo_total = matriz_custos[linhas, colunas].sum()

  return linhas, colunas, custo_total

In [ ]:
diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
# diplomatas_tratado = diplomatas_tratado['lotacao'].fillna('*', inplace=True)

In [ ]:
# diplomatas_tratado = diplomatas_tratado['lotacao'].fillna('*', inplace=True)
diplomatas_tratado

In [ ]:
pesos_arestas = []
for d, row in diplomatas_tratado.iterrows():
    pesos_diplomata = []
    for i, posto in cidades.iterrows():
        pesos_diplomata.append(calcular_custo_aresta(row,posto))
    pesos_arestas.append(pesos_diplomata)

pesos_arestas

In [ ]:
# Exemplo de uso
linhas, colunas, custo = alocar_diplomatas(pesos_arestas)
print("Alocações (diplomata -> posto):")
for i, j in zip(linhas, colunas):
  print(f"Diplomata {diplomatas_tratado.iloc[i].id} -> Posto {cidades.iloc[j].posto}")

print("Custo total:", custo)